In [1]:
from learningtools import *
import os
#os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
#os.environ["CUDA_VISIBLE_DEVICES"]="5"

In [2]:
subset = np.load("../data/simimgs.npy")
subset = subset[:, :, :, 0, :, : , :, :, :]
# (5, 3, 3, 1, 32, 5, 24, 64, 64)

# planet mass
y = np.zeros(subset.shape[:-2], dtype=int)
for i in range(5):
    y[i] = i
    
x = subset.reshape(-1, subset.shape[-4], subset.shape[-3], subset.shape[-2], subset.shape[-1])
y = y.reshape(-1, subset.shape[-4], subset.shape[-3])

# shuffle 1440 models

np.random.seed(0)
index = np.arange(np.cumprod(subset.shape)[3])
np.random.shuffle(index)

x = x[index]
y = y[index]

x = x.reshape(-1, 64, 64)
y = y.reshape(-1)

In [3]:
# agumentation
xprime = np.zeros_like(x)
for i in range(x.shape[0]):
    xnew = x[i]  + 0.1*np.std(x[i])* np.random.random( (64, 64) )
    xnew = (np.log10(xnew) - np.log10(xnew.max()) + 2.) / 2.
    xprime[i] = xnew 
xprime[xprime < 0] = 0
x = xprime

# split trainig, validation and test data to 60%, 20% and 20%.
# 0.6: 0.2: 0.2
x_train = x[:int(x.shape[0]*0.6)]
y_train = y[:int(x.shape[0]*0.6)]

x_val = x[int(x.shape[0]*0.6):int(x.shape[0]*0.8)]
y_val = y[int(x.shape[0]*0.6):int(x.shape[0]*0.8)]

x_test  = x[int(x.shape[0]*0.8):]
y_test  = y[int(x.shape[0]*0.8):]

# shuffle within the each set

index = np.arange(y_train.shape[0])
np.random.seed(0) 
np.random.shuffle(index)
x_train = x_train[index]
y_train = y_train[index]


index = np.arange(y_val.shape[0])
np.random.seed(0)
np.random.shuffle(index)
x_val = x_val[index]
y_val = y_val[index]

index = np.arange(y_test.shape[0])
np.random.seed(0)
np.random.shuffle(index)
x_test = x_test[index]
y_test = y_test[index]

In [4]:
model = tf.keras.models.Sequential([
          tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same', input_shape=( 64, 64, 1)),
          tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same'),
          tf.keras.layers.MaxPooling2D((2, 2)),
          tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
          tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
          tf.keras.layers.MaxPooling2D((2, 2)),
          tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
          tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
          tf.keras.layers.MaxPooling2D((2, 2)),
          #tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same'),
          #tf.keras.layers.Conv2D(1024, (3, 3), activation='relu', padding='same'),
          #tf.keras.layers.MaxPooling2D((2, 2)),
          tf.keras.layers.Flatten(),
          tf.keras.layers.Dense(512, activation='relu'),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Dense(512, activation='relu'),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Dense(128, activation='relu'),
          tf.keras.layers.Dropout(0.2),
          tf.keras.layers.Dense(5)
        ])

In [5]:
decay_steps = 60
initial_learning_rate = 1e-3
lr_decayed_fn = tf.keras.experimental.CosineDecay(
    initial_learning_rate, decay_steps)
def lr_schedule(x):
    return float(lr_decayed_fn(x))
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(lr_schedule)

In [ ]:
accuracies = np.zeros((2, 60))
loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    
# optimizer Adam optimization is a stochastic gradient descent method that is based on
    #  adaptive estimation of first-order and second-order moments
model.compile(tf.keras.optimizers.Adam(learning_rate=1e-3),
                  loss=loss_fn,
                  metrics=['accuracy'])


datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True)
# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
# fits the model on batches with real-time data augmentation:
model.fit(datagen.flow(x_train[:,:,:, None], y_train, batch_size=128),
          validation_data = (x_val[:,:,:, None], y_val),
                    callbacks=[lr_scheduler], epochs=60)

In [20]:
model = tf.keras.models.load_model("../model/VGG_like/model")